<a href="https://colab.research.google.com/github/UTKARSH4/Attention-Model/blob/main/Attention_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# global variables that state the filename and directory of the vocabulary file
VOCAB_FILE = 'ende_32k.subword'
VOCAB_DIR = 'data/'

# Tokenize the dataset.
tokenized_train_stream = trax.data.Tokenize(vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)(train_stream)
tokenized_eval_stream = trax.data.Tokenize(vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)(eval_stream)

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import spacy

In [2]:
config = tfds.translate.opus.OpusConfig(
    version=tfds.core.Version('0.1.0'),
    language_pair=("en", "de"),
    subsets=["GNOME"]
)
builder = tfds.builder("opus", config=config)

In [3]:
builder.download_and_prepare()

Shuffling and writing examples to /root/tensorflow_datasets/opus/de-en for GNOME/0.1.0.incompleteD4EPRK/opus-train.tfrecord


Dataset opus downloaded and prepared to /root/tensorflow_datasets/opus/de-en for GNOME/0.1.0. Subsequent calls will reuse this data.


In [4]:
 ds_train= builder.as_dataset(split='train', shuffle_files=True)
#ds_test= builder.as_dataset(split='test', shuffle_files=True)

In [5]:
data_set=tfds.as_dataframe(ds_train)

In [6]:
data_set["en"]=data_set["en"].apply(lambda x:x.split())
data_set["de"]=data_set["de"].apply(lambda x:x.split())


In [9]:
length_en=max([len(x) for x in data_set["en"] ])
length_de=max([len(x) for x in data_set["en"] ])

In [15]:
vocab_size = 55000
embedding_dim = 16
max_length = 130 # Expected max_len is 120 a few cells down.
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_portion = .8

In [12]:
for i in range(data_set.shape[0]):
  data_set["en"][i].append("[EOS]")
  data_set["de"][i].append("[EOS]")

In [13]:
tokenizer_src =Tokenizer(oov_token="<OOV>")
tokenizer_src.fit_on_texts(data_set["en"])
word_index = tokenizer_src.word_index

train_sequences = tokenizer_src.texts_to_sequences(data_set["en"])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')


#eval_sequences = tokenizer_text.texts_to_sequences(eval_set.merged)
#eval_padded = pad_sequences(eval_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [14]:
tokenizer_trgt =Tokenizer(oov_token="<OOV>")
tokenizer_trgt.fit_on_texts(data_set["de"])
word_index_trgt = tokenizer_trgt.word_index

train_sequences_trgt = tokenizer_trgt.texts_to_sequences(data_set["de"])
train_padded = pad_sequences(train_sequences_trgt, maxlen=max_length, padding='post', truncating='post')


#eval_sequences = tokenizer_text.texts_to_sequences(eval_set.merged)
#eval_padded = pad_sequences(eval_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
def encoder(vocab_size,embedding_dim,max_length,layers):
  model_encoder = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
      tf.keras.layers.Attention(),
      tf.keras.layers.LSTM(24, activation='relu')*layers
  ])
  return model_encoder

In [ ]:
def decoder(vocab_size,embedding_dim,max_length):
  model_decoder = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
      tf.keras.layers.LSTM(24, activation='relu')
  ])
  return model_decoder

In [16]:
def attenion_inputs(encoder_activations, decoder_activations, inputs):
  keys = encoder_activations
  value = encoder_activations
  query = decoder_activations
  mask = inputs != 0
  mask = np.reshape(mask, 1,mask[0])
  mask = mask + np.zeros((decoder_activations.shape[1], 1))

  return query, keys, value, mask



In [ ]:
def AttentionQKV(d_feature, n_heads=1, dropout=0.0, mode='train'):
  return tf.keras.Sequential([tf.keras.layers.concatenate([tf.keras.layers.Dense(d_feature),
                                                           tf.keras.layers.Dense(d_feature),
                                                           tf.keras.layers.Dense(d_feature)]),
                              tf.keras.layers.Attention(),
                              tf.keras.layers.Dense(d_feature)])



In [ ]:
def NMTAttn(vocab_size,
            d_model=1024,
            n_encoder_layers=2,
            n_decoder_layers=2,
            n_attention_heads=4,
            attention_dropout=0.0,
            mode='train'):
  input_encoder = encoder(vocab_size,d_model,max_length,n_encoder_layers);
  pre_attention_decoder = decoder(vocab_size,d_model,max_length)

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)